In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import node2vec

import torch
import torch_geometric as tg

from sklearn.manifold import SpectralEmbedding
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split

## Read Data

### AIDS Data
https://networkrepository.com/AIDS.php

In [2]:
AIDS_EDGES = None
AIDS_LABELS = None

with open('data/AIDS/AIDS.edges.txt') as edges_file:
    AIDS_EDGES = np.loadtxt(edges_file, dtype=int, delimiter=',')

with open('data/AIDS/AIDS.node_labels.txt') as nodes_file:
    AIDS_LABELS = np.loadtxt(nodes_file, dtype=int, delimiter=',')
    AIDS_LABELS = AIDS_LABELS[:, 1]

### FB CMU Data
https://networkrepository.com/fb-CMU-Carnegie49.php

In [3]:
FB_CMU_EDGES = None
FB_CMU_LABELS = None

with open('data/fb-CMU-Carnegie49/fb-CMU-Carnegie49.edges.txt') as edges_file:
    FB_CMU_EDGES = np.loadtxt(edges_file, dtype=int, delimiter=' ')

with open('data/fb-CMU-Carnegie49/fb-CMU-Carnegie49.node_labels.txt') as nodes_file:
    FB_CMU_LABELS = np.loadtxt(nodes_file, dtype=int, delimiter=',')
    FB_CMU_LABELS = FB_CMU_LABELS[:, 1]

## Web Spam Detection Data
https://networkrepository.com/web-spam-detection.php

In [4]:
SPAM_EDGES = None
SPAM_LABELS = None

with open('data/web-spam-detection/web-spam-detection.edges.txt') as edges_file:
    SPAM_EDGES = np.loadtxt(edges_file, dtype=int, delimiter=',')

with open('data/web-spam-detection/web-spam-detection.node_labels.txt') as nodes_file:
    SPAM_LABELS = np.loadtxt(nodes_file, dtype=int, delimiter=',')
    SPAM_LABELS = SPAM_LABELS[:, 1]

## Data Preprocessing

In [5]:
# X = AIDS_EDGES
# Y = AIDS_LABELS

X = FB_CMU_EDGES
Y = FB_CMU_LABELS

# X = SPAM_EDGES
# Y = SPAM_LABELS

In [6]:
G = nx.DiGraph()
G.add_edges_from(X)

In [7]:
EPOCHS = 10000
NUM_NODES = np.max(X)
NUM_CLASSES_CLUSTERS = np.max(Y)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
train_mask = torch.zeros(NUM_NODES, dtype=torch.bool)
test_mask = torch.zeros(NUM_NODES, dtype=torch.bool)
train_mask[:int(NUM_NODES*0.8)] = True
test_mask[int(NUM_NODES*0.8):] = True

data = tg.data.Data(
    train_mask=train_mask,
    test_mask=test_mask,
    x=torch.eye(NUM_NODES), 
    edge_index=torch.tensor(X-1, dtype=torch.long).t().contiguous(), 
    y=torch.nn.functional.one_hot(torch.tensor(Y-1, dtype=torch.long), NUM_CLASSES_CLUSTERS)).to(device)
    # y=torch.tensor(Y-1, dtype=torch.long)).to(device)

In [9]:
adjacency_matrix = np.zeros((NUM_NODES, NUM_NODES))
adjacency_matrix[X[:, 0]-1, X[:, 1]-1] = 1
adjacency_matrix[X[:, 1]-1, X[:, 0]-1] = 1

## Fully Connected Neural Network

In [10]:
class FCN(torch.nn.Module):
    def __init__(self, num_nodes, num_classes):
        super(FCN, self).__init__()
        self.fc1 = torch.nn.Linear(num_nodes, 512)
        self.fc2 = torch.nn.Linear(512, 128)
        self.fc3 = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.softmax(self.fc3(x), dim=1)
        return x

In [11]:
model_fcn = FCN(NUM_NODES, NUM_CLASSES_CLUSTERS).to(device)
optimizer_fcn = torch.optim.Adam(model_fcn.parameters(), lr=0.001)# , weight_decay=5e-4)
loss_fn_fcn = torch.nn.MSELoss()

In [12]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(adjacency_matrix, Y, test_size=0.2, random_state=42)
X_TRAIN = torch.tensor(X_TRAIN, dtype=torch.float).to(device)
X_TEST = torch.tensor(X_TEST, dtype=torch.float).to(device)
Y_TRAIN = torch.nn.functional.one_hot(torch.tensor(Y_TRAIN-1, dtype=torch.long)).to(device)
Y_TEST = torch.nn.functional.one_hot(torch.tensor(Y_TEST-1, dtype=torch.long)).to(device)

In [13]:
loss_list_fcn = []
acc_list_fcn = []

for epoch in range(EPOCHS):
    model_fcn.train()
    optimizer_fcn.zero_grad()
    out = model_fcn(X_TRAIN)
    loss = loss_fn_fcn(out, Y_TRAIN.float())
    loss.backward()
    optimizer_fcn.step()

    model_fcn.eval()
    _, pred = model_fcn(X_TEST).max(dim=1)
    correct = float(pred.eq(Y_TEST.float().argmax(dim=1)).sum().item())
    acc = correct / len(Y_TEST)
    if (epoch+1) % 100 == 0:
        print('Epoch: {:03d}, Loss: {:.5f}, Acc: {:.5f}'.format(epoch+1, loss, acc))

    loss_list_fcn.append(loss.item())
    acc_list_fcn.append(acc)

Epoch: 100, Loss: 0.00632, Acc: 0.74548
Epoch: 200, Loss: 0.00593, Acc: 0.74849
Epoch: 300, Loss: 0.00591, Acc: 0.75075
Epoch: 400, Loss: 0.00591, Acc: 0.75000
Epoch: 500, Loss: 0.00591, Acc: 0.75075
Epoch: 600, Loss: 0.00590, Acc: 0.75075
Epoch: 700, Loss: 0.00590, Acc: 0.75000
Epoch: 800, Loss: 0.00565, Acc: 0.75000
Epoch: 900, Loss: 0.00565, Acc: 0.74774
Epoch: 1000, Loss: 0.00565, Acc: 0.74925
Epoch: 1100, Loss: 0.00565, Acc: 0.74925
Epoch: 1200, Loss: 0.00565, Acc: 0.74849
Epoch: 1300, Loss: 0.00565, Acc: 0.74849
Epoch: 1400, Loss: 0.00565, Acc: 0.74774
Epoch: 1500, Loss: 0.00565, Acc: 0.74774
Epoch: 1600, Loss: 0.00565, Acc: 0.74849
Epoch: 1700, Loss: 0.00565, Acc: 0.74849
Epoch: 1800, Loss: 0.00565, Acc: 0.74849
Epoch: 1900, Loss: 0.00565, Acc: 0.75000
Epoch: 2000, Loss: 0.00565, Acc: 0.74925
Epoch: 2100, Loss: 0.00565, Acc: 0.74925
Epoch: 2200, Loss: 0.00565, Acc: 0.74699
Epoch: 2300, Loss: 0.00565, Acc: 0.74699
Epoch: 2400, Loss: 0.00565, Acc: 0.74623
Epoch: 2500, Loss: 0.0056

In [14]:
# test the model
model_fcn.eval()
_, pred = model_fcn(X_TEST).max(dim=1)
correct = float(pred.eq(Y_TEST.argmax(dim=1)).sum().item())
acc = correct / len(Y_TEST)
print('Test Accuracy: {:.5f}'.format(acc))
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST.cpu().numpy().argmax(axis=1), pred.cpu().numpy())}")

Test Accuracy: 0.74623
Adjusted Rand Index: 0.3666679314165358


In [15]:
# plot loss and accuracy
plt.plot(loss_list_fcn)
plt.plot(acc_list_fcn)
plt.legend(['loss', 'accuracy'])
plt.show()

## Convolutional Neural Network

```
class CNN(torch.nn.Module):
    def __init__(self, num_nodes, num_classes):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=2)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(128*10*10, 1024)
        self.fc2 = torch.nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = self.pool(torch.nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128*10*10)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.softmax(self.fc2(x), dim=1)
        return x
```

```
model_cnn = CNN(NUM_NODES, NUM_CLASSES_CLUSTERS).to(device)
optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=0.1, weight_decay=5e-4)
loss_fn_cnn = torch.nn.MSELoss()
```

```
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(adjacency_matrix, Y, test_size=0.2, random_state=42)
X_TRAIN = torch.tensor(X_TRAIN, dtype=torch.float).to(device)
X_TEST = torch.tensor(X_TEST, dtype=torch.float).to(device)
Y_TRAIN = torch.nn.functional.one_hot(torch.tensor(Y_TRAIN-1, dtype=torch.long)).to(device)
Y_TEST = torch.nn.functional.one_hot(torch.tensor(Y_TEST-1, dtype=torch.long)).to(device)
```

```
X_TRAIN = X_TRAIN.view(-1, 1, 49, 49)
X_TEST = X_TEST.view(-1, 1, 49, 49)

dataset = torch.utils.data.TensorDataset(X_TRAIN, Y_TRAIN)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
```

```
for epoch in range(EPOCHS):
    for data, label in dataloader:
        model_cnn.train()
        optimizer_cnn.zero_grad()
        out = model_cnn(data)
        loss = loss_fn_cnn(out, label.float())
        loss.backward()
        optimizer_cnn.step()
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch+1, loss))
```

```
loss_list_cnn = []
acc_list_cnn = []

for epoch in range(EPOCHS):
    model_cnn.train()
    optimizer_cnn.zero_grad()
    out = model_cnn(X_TRAIN)
    loss = loss_fn_fcn(out, Y_TRAIN.float())
    loss.backward()
    optimizer_cnn.step()

    model_cnn.eval()
    _, pred = model_cnn(X_TEST).max(dim=1)
    correct = float(pred.eq(Y_TEST.float().argmax(dim=1)).sum().item())
    acc = correct / len(Y_TEST)
    if (epoch+1) % 100 == 0:
        print('Epoch: {:03d}, Loss: {:.5f}, Acc: {:.5f}'.format(epoch+1, loss, acc))

    loss_list_fcn.append(loss.item())
    acc_list_fcn.append(acc)
```

```
# test the model
model_cnn.eval()
_, pred = model_cnn(X_TEST).max(dim=1)
correct = float(pred.eq(Y_TEST.argmax(dim=1)).sum().item())
acc = correct / len(data.y)
print('Test Accuracy: {:.5f}'.format(acc))
```

```
# plot loss and accuracy
plt.plot(loss_list_cnn)
plt.plot(acc_list_cnn)
plt.legend(['loss', 'accuracy'])
plt.show()
```

## Graph Convolutional Network with 2 fully connected layers 

In [16]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = tg.nn.GCNConv(num_node_features, 1024)
        self.conv2 = tg.nn.GCNConv(1024, 256)
        self.conv3 = tg.nn.GCNConv(256, 128)
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, num_classes)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        # x = torch.nn.functional.relu(x)
        x = torch.nn.functional.softmax(x, dim=1)
        return x

In [17]:
model_gcn = GCN(num_node_features=NUM_NODES, num_classes=NUM_CLASSES_CLUSTERS).to(device)
optimizer_gcn = torch.optim.Adam(model_gcn.parameters(), lr=0.001, weight_decay=5e-4)
loss_fn_gcn = torch.nn.CrossEntropyLoss()

In [18]:
loss_list_gcn = []
accuracy_list_gcn = []

for epoch in range(EPOCHS):
    model_gcn.train()
    optimizer_gcn.zero_grad()
    out = model_gcn(data)
    loss = loss_fn_gcn(out[data.train_mask], data.y[data.train_mask].float())
    loss.backward()
    optimizer_gcn.step()

    # validate the model
    model_gcn.eval()
    _, pred = model_gcn(data).max(dim=1)
    _, true_labels = torch.max(data.y.float(), dim=1)
    correct = float(pred[data.test_mask].eq(true_labels[data.test_mask]).sum().item())
    acc = correct / len(data.y[data.test_mask])
    loss_list_gcn.append(loss.item())
    accuracy_list_gcn.append(acc)
    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1:03d}\t Loss: {loss:.4f}\t Val Acc: {acc:.4f}')

Epoch: 100	 Loss: 0.9512	 Val Acc: 0.5836
Epoch: 200	 Loss: 0.8471	 Val Acc: 0.6551
Epoch: 300	 Loss: 0.8309	 Val Acc: 0.6837
Epoch: 400	 Loss: 0.8236	 Val Acc: 0.6694
Epoch: 500	 Loss: 0.8305	 Val Acc: 0.6807
Epoch: 600	 Loss: 0.8167	 Val Acc: 0.6785
Epoch: 700	 Loss: 0.8144	 Val Acc: 0.6837
Epoch: 800	 Loss: 0.8026	 Val Acc: 0.6777
Epoch: 900	 Loss: 0.8033	 Val Acc: 0.6830
Epoch: 1000	 Loss: 0.8241	 Val Acc: 0.6770
Epoch: 1100	 Loss: 0.8075	 Val Acc: 0.6777
Epoch: 1200	 Loss: 0.8030	 Val Acc: 0.6755
Epoch: 1300	 Loss: 0.7954	 Val Acc: 0.6574
Epoch: 1400	 Loss: 0.7868	 Val Acc: 0.6890
Epoch: 1500	 Loss: 0.8108	 Val Acc: 0.6920
Epoch: 1600	 Loss: 0.7901	 Val Acc: 0.6679
Epoch: 1700	 Loss: 0.7856	 Val Acc: 0.6867
Epoch: 1800	 Loss: 0.7798	 Val Acc: 0.6898
Epoch: 1900	 Loss: 0.7849	 Val Acc: 0.6453
Epoch: 2000	 Loss: 0.7849	 Val Acc: 0.6845
Epoch: 2100	 Loss: 0.7784	 Val Acc: 0.6995
Epoch: 2200	 Loss: 0.7807	 Val Acc: 0.6935
Epoch: 2300	 Loss: 0.7756	 Val Acc: 0.6905
Epoch: 2400	 Loss: 0

In [19]:
# test the model
model_gcn.eval()
_, pred = model_gcn(data).max(dim=1)
_, true_labels = torch.max(data.y.float(), dim=1)
correct = float(pred[data.test_mask].eq(true_labels[data.test_mask]).sum().item())
acc = correct / len(data.y[data.test_mask])
print(f'Test Acc: {acc:.4f}')
print(f"Adjusted Rand Index: {adjusted_rand_score(pred.cpu().numpy(), true_labels.cpu().numpy())}")

Test Acc: 0.6958
Adjusted Rand Index: 0.47229196928965944


In [20]:
# plot loss and accuracy
plt.plot(loss_list_gcn)
plt.plot(accuracy_list_gcn)
plt.legend(['loss', 'accuracy'])
plt.show()

## Graph Encoder Embedding

In [21]:
class GraphEncoderEmbedding(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GraphEncoderEmbedding, self).__init__()
        self.conv1 = tg.nn.GCNConv(num_features, 256)
        self.conv2 = tg.nn.GCNConv(256, num_classes)

    def forward(self, x, edge_index):
        
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = torch.nn.functional.softmax(x, dim=1)
        return x

In [22]:
model_gcn_lda = GraphEncoderEmbedding(num_features=NUM_NODES, num_classes=NUM_CLASSES_CLUSTERS).to(device)
optimizer_gcn_lda = torch.optim.Adam(model_gcn_lda.parameters(), lr=0.01) #, weight_decay=5e-4)
loss_fn_gcn_lda = torch.nn.MSELoss()

In [23]:
loss_list_gcn_lda = []
accuracy_list_gcn_lda = []

for epoch in range(EPOCHS):
    model_gcn_lda.train()
    optimizer_gcn_lda.zero_grad()
    out = model_gcn_lda(data.x, data.edge_index)
    loss = loss_fn_gcn_lda(out[data.train_mask], data.y[data.train_mask].float())
    loss.backward()
    optimizer_gcn_lda.step()

    # validate the model
    model_gcn_lda.eval()
    _, pred = model_gcn_lda(data.x, data.edge_index).max(dim=1)
    _, true_labels = torch.max(data.y.float(), dim=1)
    correct = float(pred[data.test_mask].eq(true_labels[data.test_mask]).sum().item())
    acc = correct / len(data.y[data.test_mask])
    loss_list_gcn_lda.append(loss.item())
    accuracy_list_gcn_lda.append(acc)
    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1:03d}\t Loss: {loss:.4f}\t Val Acc: {acc:.4f}')

Epoch: 100	 Loss: 0.0623	 Val Acc: 0.7123
Epoch: 200	 Loss: 0.0395	 Val Acc: 0.7041
Epoch: 300	 Loss: 0.0304	 Val Acc: 0.6943
Epoch: 400	 Loss: 0.0253	 Val Acc: 0.6913
Epoch: 500	 Loss: 0.0222	 Val Acc: 0.6913
Epoch: 600	 Loss: 0.0206	 Val Acc: 0.6837
Epoch: 700	 Loss: 0.0197	 Val Acc: 0.6845
Epoch: 800	 Loss: 0.0190	 Val Acc: 0.6837
Epoch: 900	 Loss: 0.0182	 Val Acc: 0.6822
Epoch: 1000	 Loss: 0.0179	 Val Acc: 0.6792
Epoch: 1100	 Loss: 0.0174	 Val Acc: 0.6860
Epoch: 1200	 Loss: 0.0175	 Val Acc: 0.6822
Epoch: 1300	 Loss: 0.0169	 Val Acc: 0.6770
Epoch: 1400	 Loss: 0.0169	 Val Acc: 0.6845
Epoch: 1500	 Loss: 0.0167	 Val Acc: 0.6770
Epoch: 1600	 Loss: 0.0165	 Val Acc: 0.6800
Epoch: 1700	 Loss: 0.0164	 Val Acc: 0.6792
Epoch: 1800	 Loss: 0.0164	 Val Acc: 0.6777
Epoch: 1900	 Loss: 0.0165	 Val Acc: 0.6762
Epoch: 2000	 Loss: 0.0164	 Val Acc: 0.6792
Epoch: 2100	 Loss: 0.0163	 Val Acc: 0.6770
Epoch: 2200	 Loss: 0.0164	 Val Acc: 0.6717
Epoch: 2300	 Loss: 0.0161	 Val Acc: 0.6755
Epoch: 2400	 Loss: 0

In [24]:
# test the model
model_gcn_lda.eval()
_, pred = model_gcn_lda(data.x, data.edge_index).max(dim=1)
_, true_labels = torch.max(data.y.float(), dim=1)
correct = float(pred[data.test_mask].eq(true_labels[data.test_mask]).sum().item())
acc = correct / len(data.y[data.test_mask])
print(f'Test Acc: {acc:.4f}')

Test Acc: 0.6702


In [25]:
# plot loss and accuracy
plt.plot(loss_list_gcn_lda)
plt.plot(accuracy_list_gcn_lda)
plt.legend(['loss', 'accuracy'])
plt.show()

In [26]:
model_gcn_lda.eval()
with torch.no_grad():
    embeddings = model_gcn_lda(data.x, data.edge_index)

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(embeddings.cpu().numpy(), Y, test_size=0.2, random_state=42)

lda_gcn = LinearDiscriminantAnalysis()
lda_gcn.fit(X_TRAIN, Y_TRAIN)
Y_PRED_LDA_GCN = lda_gcn.predict(X_TRAIN)

print("Training Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_LDA_GCN) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_LDA_GCN)}")

print()
print("Testing Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TEST == lda_gcn.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, lda_gcn.predict(X_TEST))}")

Training Accuracy LDA:
Accuracy: 0.9133546807308345
Adjusted Rand Index: 0.743561265746264

Testing Accuracy LDA:
Accuracy: 0.9254518072289156
Adjusted Rand Index: 0.7807758894008909


## Spectral Embedding

In [27]:
embedding = SpectralEmbedding(n_components=NUM_CLASSES_CLUSTERS)  # 2D embedding
spectral_embedding = embedding.fit_transform(adjacency_matrix, Y)

In [28]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(spectral_embedding, Y, test_size=0.2, random_state=42)

In [29]:
lda_spectral = LinearDiscriminantAnalysis()
lda_spectral.fit(X_TRAIN, Y_TRAIN)
Y_PRED_LDA_SPECTRAL = lda_spectral.predict(X_TRAIN)

print("Training Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_LDA_SPECTRAL) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_LDA_SPECTRAL)}")

print()
print("Testing Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TEST == lda_spectral.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, lda_spectral.predict(X_TEST))}")

Training Accuracy LDA:
Accuracy: 0.5503861367489169
Adjusted Rand Index: 0.0020506684744887103

Testing Accuracy LDA:
Accuracy: 0.5466867469879518
Adjusted Rand Index: -0.0003962629562524497


In [30]:
qda_spectral = QuadraticDiscriminantAnalysis()
qda_spectral.fit(X_TRAIN, Y_TRAIN)
Y_PRED_QDA_SPECTRAL = qda_spectral.predict(X_TRAIN)

print("Training Accuracy QDA:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_QDA_SPECTRAL) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_QDA_SPECTRAL)}")

print()
print("Testing Accuracy QDA:")
print(f"Accuracy: {np.sum(Y_TEST == qda_spectral.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, qda_spectral.predict(X_TEST))}")

Training Accuracy QDA:
Accuracy: 0.5377660576379732
Adjusted Rand Index: -0.0052523129906634535

Testing Accuracy QDA:
Accuracy: 0.5406626506024096
Adjusted Rand Index: -0.0021172162511052858


In [31]:
kmeans_spectral = KMeans(n_clusters=NUM_CLASSES_CLUSTERS)
kmeans_spectral.fit(X_TRAIN)
Y_PRED_KMEANS_SPECTRAL = kmeans_spectral.predict(X_TRAIN)

print("Training Accuracy KMeans:")
print(f"Accuracy: {np.sum(Y_TRAIN-1 == Y_PRED_KMEANS_SPECTRAL) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN-1, Y_PRED_KMEANS_SPECTRAL)}")

print()
print("Testing Accuracy KMeans:")
print(f"Accuracy: {np.sum(Y_TEST-1 == kmeans_spectral.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST-1, kmeans_spectral.predict(X_TEST))}")

Training Accuracy KMeans:
Accuracy: 0.09361461668864193
Adjusted Rand Index: -0.00287458294196429

Testing Accuracy KMeans:
Accuracy: 0.10316265060240964
Adjusted Rand Index: -0.000558088228946664


c:\Users\Maitri\Desktop\STAT622\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [32]:
svm_spectral = SVC()
svm_spectral.fit(X_TRAIN, Y_TRAIN)
Y_PRED_SVM_SPECTRAL = svm_spectral.predict(X_TRAIN)

print("Training Accuracy SVM:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_SVM_SPECTRAL) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_SVM_SPECTRAL)}")

print()
print("Testing Accuracy SVM:")
print(f"Accuracy: {np.sum(Y_TEST == svm_spectral.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, svm_spectral.predict(X_TEST))}")

Training Accuracy SVM:
Accuracy: 0.562817856470145
Adjusted Rand Index: 0.012879884293737308

Testing Accuracy SVM:
Accuracy: 0.5587349397590361
Adjusted Rand Index: 0.006921547161471075


## Node2Vec

In [33]:
node2vec_model = node2vec.Node2Vec(G, dimensions=NUM_CLASSES_CLUSTERS, num_walks=200, workers=4)

model = node2vec_model.fit(window=10, min_count=1, batch_words=4)

embeddings = model.wv

node_ids = list(G.nodes())  # List of nodes in the graph
embedding_matrix = np.array([model.wv[str(node_id)] for node_id in node_ids])

Computing transition probabilities:   0%|          | 0/6637 [00:00<?, ?it/s]

In [34]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(embedding_matrix, Y, test_size=0.2, random_state=42)

In [35]:
lda_node = LinearDiscriminantAnalysis(n_components=2)
lda_node.fit(X_TRAIN, Y_TRAIN)
Y_PRED_LDA_NODE = lda_node.predict(X_TRAIN)

print("Training Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_LDA_NODE) / len(Y)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_LDA_NODE)}")\

print()
print("Testing Accuracy LDA:")
print(f"Accuracy: {np.sum(Y_TEST == lda_node.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, lda_node.predict(X_TEST))}")

Training Accuracy LDA:
Accuracy: 0.43995781226457736
Adjusted Rand Index: 0.0

Testing Accuracy LDA:
Accuracy: 0.552710843373494
Adjusted Rand Index: 0.0


In [36]:
qda_node = QuadraticDiscriminantAnalysis()
qda_node.fit(X_TRAIN, Y_TRAIN)
Y_PRED_QDA_NODE = qda_node.predict(X_TRAIN)

print("Training Accuracy QDA:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_QDA_NODE) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_QDA_NODE)}")

print()
print("Testing Accuracy QDA:")
print(f"Accuracy: {np.sum(Y_TEST == qda_node.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, qda_node.predict(X_TEST))}")

Training Accuracy QDA:
Accuracy: 0.5500094179694858
Adjusted Rand Index: 0.0

Testing Accuracy QDA:
Accuracy: 0.552710843373494
Adjusted Rand Index: 0.0


In [37]:
kmeans_node = KMeans(n_clusters=NUM_CLASSES_CLUSTERS)
kmeans_node.fit(X_TRAIN)
Y_PRED_KMEANS_NODE = kmeans_node.predict(X_TRAIN)

print("Training Accuracy KMeans:")
print(f"Accuracy: {np.sum(Y_TRAIN-1 == Y_PRED_KMEANS_NODE) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN-1, Y_PRED_KMEANS_NODE)}")

print()
print("Testing Accuracy KMeans:")
print(f"Accuracy: {np.sum(Y_TEST-1 == kmeans_node.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST-1, kmeans_node.predict(X_TEST))}")

Training Accuracy KMeans:
Accuracy: 0.23620267470333395
Adjusted Rand Index: 0.0005141761950523399

Testing Accuracy KMeans:
Accuracy: 0.2289156626506024
Adjusted Rand Index: 0.001062994366612492


c:\Users\Maitri\Desktop\STAT622\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [38]:
svm_node = SVC()
svm_node.fit(X_TRAIN, Y_TRAIN)
Y_PRED_SVM_NODE = svm_node.predict(X_TRAIN)

print("Training Accuracy SVM:")
print(f"Accuracy: {np.sum(Y_TRAIN == Y_PRED_SVM_NODE) / len(Y_TRAIN)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TRAIN, Y_PRED_SVM_NODE)}")

print()
print("Testing Accuracy SVM:")
print(f"Accuracy: {np.sum(Y_TEST == svm_node.predict(X_TEST)) / len(Y_TEST)}")
print(f"Adjusted Rand Index: {adjusted_rand_score(Y_TEST, svm_node.predict(X_TEST))}")

Training Accuracy SVM:
Accuracy: 0.5501977773592014
Adjusted Rand Index: 0.00010353942681341724

Testing Accuracy SVM:
Accuracy: 0.552710843373494
Adjusted Rand Index: 0.0
